# 📥 FASE 1: Extracción Automática (WOL)

In [ ]:
# =============================================================================
#  🤖 ASISTENTE DE PROGRAMACIÓN VMC - FASE 1: EXTRACCIÓN DE DATOS (WOL)
# =============================================================================
#  DESCRIPCIÓN GENERAL:
#  Este programa se conecta a la Biblioteca en Línea (WOL), descarga la info
#  de las reuniones del mes seleccionado y la guarda en tu hoja 'Info-reunion'.
#  Funciona de manera interactiva pidiendo Año y Mes.
# =============================================================================
#  🛠️ ÚLTIMAS CORRECCIONES APLICADAS (ENERO 2026):
#  1. Limpieza automática: Borra datos viejos de 'Info-reunion' antes de escribir
#     para evitar duplicados al final de la hoja.
#  2. Mapeo estricto: Se fuerza el orden de las columnas para evitar que la
#     'Lectura de la Biblia' aparezca en 'Maestros Título 1'.
# =============================================================================

import requests
from bs4 import BeautifulSoup
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import re
import calendar
from datetime import datetime, timedelta
import time
import os

# Detectar entorno para Google Colab
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    pass

# ==========================================
# 1. CONFIGURACIÓN
# ==========================================

RUTA_ARCHIVO = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/Programación VMC_Septiembre-2025-2026.xlsx'

# DEFINIMOS EL ORDEN EXACTO DE LAS COLUMNAS (Para solucionar el desorden en excel)
COLUMNAS_ORDENADAS = [
    'Semana',
    'Libro',
    'Canción Inicial',
    'Tesoros de la Biblia',
    'Segunda Canción',
    'Tercera Canción',
    'Maestros Título 1',
    'Maestros Título 2',
    'Maestros Título 3',
    'Maestros Título 4',
    'NVC Título 1',
    'NVC Título 2',
    'NVC Título 3',
    'Info Lectura Biblia',
    'Info Estudio Libro'
]

def obtener_fecha_interactiva():
    print("📅 CONFIGURACIÓN DE BÚSQUEDA")
    print("----------------------------")
    try:
        anio = int(input("👉 Ingresa el AÑO (ej. 2026): "))
        mes = int(input("👉 Ingresa el MES (1-12): "))
        return anio, mes
    except ValueError:
        return None, None

# ==========================================
# 2. GENERADOR DE URLs
# ==========================================

def generar_urls_dinamicas(anio, mes):
    print(f"\n🔍 Calculando semanas para: {mes}/{anio}...")
    urls = []
    c = calendar.Calendar(firstweekday=calendar.MONDAY)
    monthcal = c.monthdatescalendar(anio, mes)

    for week in monthcal:
        monday = week[0]
        # Si el lunes cae en el mes, generamos la URL
        if monday.month == mes:
            # Usamos el enlace del texto diario que redirige a la reunión
            url = f"https://wol.jw.org/es/wol/dt/r4/lp-s/{monday.year}/{monday.month:02d}/{monday.day:02d}"
            if url not in urls:
                urls.append(url)
    return urls

# ==========================================
# 3. EXTRACCIÓN (NÚCLEO)
# ==========================================

def limpiar_texto_estudio(texto):
    if not texto: return ""
    libros = ['lfb', 'bt', 'lmd', 'lvs', 'cf', 'rr', 'ia', 'jr']
    texto_limpio = texto
    for libro in libros:
        patron = rf"\b{libro}(?=[a-zA-Z0-9])"
        texto_limpio = re.sub(patron, f"{libro} ", texto_limpio, flags=re.IGNORECASE)
    return texto_limpio

def extraer_informacion(url):
    HEADERS = {'User-Agent': 'Mozilla/5.0'}
    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # Inicializamos el diccionario con las claves en orden correcto y vacías
        info = {col: "" for col in COLUMNAS_ORDENADAS}

        # Variables temporales para listas
        titulos_maestros = []
        nvc_titulos = []

        # --- A. Datos Básicos ---
        semana_elem = soup.find('h1', id='p1')
        info['Semana'] = semana_elem.get_text(strip=True) if semana_elem else "Semana desconocida"

        libro_elem = soup.find('h2', id='p2')
        if libro_elem:
            strongs = libro_elem.find_all('strong')
            info['Libro'] = " ".join([s.get_text(strip=True) for s in strongs]) if strongs else libro_elem.get_text(strip=True)

        # --- B. Tesoros ---
        posibles = soup.find_all(lambda tag: tag.name in ['h3', 'p'] and "(10 mins.)" in tag.text)
        for elem in posibles:
            if "Perlas escondidas" in elem.text: continue
            clean = elem.get_text(strip=True).replace("(10 mins.)", "").strip()
            if clean:
                info['Tesoros de la Biblia'] = clean
                break

        if not info['Tesoros de la Biblia']:
            elem_p5 = soup.find(id='p5')
            if elem_p5: info['Tesoros de la Biblia'] = elem_p5.get_text(strip=True).replace("(10 mins.)", "").strip()

        # --- C. Canciones ---
        canciones = []
        for h3 in soup.find_all('h3'):
            txt = h3.get_text(strip=True)
            if "Canción" in txt:
                m = re.search(r'Canción\s+(\d+)', txt)
                if m: canciones.append(f"Canción {m.group(1)}")

        if len(canciones) > 0: info['Canción Inicial'] = canciones[0]
        if len(canciones) > 1: info['Segunda Canción'] = canciones[1]
        if len(canciones) > 2: info['Tercera Canción'] = canciones[2]

        # --- D. Lectura y Estudio ---
        parrafos = soup.find_all('p')
        for p in parrafos:
            txt = p.get_text(strip=True)
            # Lectura (Busca "4 mins" o similar)
            if "(4 mins.)" in txt and not info['Info Lectura Biblia']:
                 m = re.search(r'\((?:4|3)\s+mins?.*?\)\s*(.+)', txt)
                 if m: info['Info Lectura Biblia'] = m.group(1).strip()
            # Estudio (Busca "30 mins")
            if "(30 mins.)" in txt and not info['Info Estudio Libro']:
                m = re.search(r'\(30\s+mins?.*?\)\s*(.+)', txt)
                if m: info['Info Estudio Libro'] = limpiar_texto_estudio(m.group(1).strip())

        # --- E. Maestros y Vida (Escaneo Secuencial) ---
        seccion = None
        headers = soup.find_all(['h2', 'h3'])

        for h in headers:
            txt = h.get_text(strip=True)

            if h.name == 'h2':
                if "SEAMOS MEJORES MAESTROS" in txt.upper(): seccion = "MAESTROS"
                elif "NUESTRA VIDA CRISTIANA" in txt.upper(): seccion = "VIDA"
                elif "TESOROS" not in txt.upper(): pass

            elif h.name == 'h3' and seccion:
                if "Canción" in txt or "Palabras de conclusión" in txt: continue
                if "Artículo de estudio" in txt: break

                if seccion == "MAESTROS":
                    titulos_maestros.append(txt)
                elif seccion == "VIDA":
                    # Si detectamos Estudio Bíblico o Conclusión, no lo agregamos como título NVC normal
                    if "Estudio bíblico de la congregación" in txt:
                         pass
                    else:
                        nvc_titulos.append(txt)

        # Asignar Maestros a columnas específicas
        for i, titulo in enumerate(titulos_maestros):
            if i < 4: info[f'Maestros Título {i+1}'] = titulo

        # Asignar NVC
        if len(nvc_titulos) > 0: info['NVC Título 1'] = nvc_titulos[0]
        if len(nvc_titulos) > 1: info['NVC Título 2'] = nvc_titulos[1]

        # El NVC 3 siempre suele ser el Estudio
        info['NVC Título 3'] = "Estudio bíblico de la congregación"

        return info

    except Exception as e:
        print(f"❌ Error leyendo {url}: {e}")
        return None

# ==========================================
# 4. GUARDADO (LIMPIEZA + ESCRITURA)
# ==========================================

def guardar_excel_limpio(datos, ruta):
    print(f"\n💾 Actualizando archivo: {os.path.basename(ruta)}...")
    try:
        book = load_workbook(ruta)
        if 'Info-reunion' not in book.sheetnames:
            book.create_sheet('Info-reunion')

        ws = book['Info-reunion']

        # 1. LIMPIEZA: Borrar contenido desde la fila 2 hacia abajo
        num_filas = ws.max_row
        if num_filas > 1:
            print(f"   🧹 Limpiando {num_filas-1} filas antiguas...")
            ws.delete_rows(2, amount=num_filas-1)

        # 2. ESCRITURA: Escribir los nuevos datos
        print(f"   ✍️ Escribiendo {len(datos)} semanas nuevas...")

        # Convertimos la lista de diccionarios a DataFrame asegurando el orden
        df_new = pd.DataFrame(datos, columns=COLUMNAS_ORDENADAS)

        rows = dataframe_to_rows(df_new, index=False, header=False)
        for r_idx, row in enumerate(rows, 1):
            for c_idx, value in enumerate(row, 1):
                # Escribimos en fila r_idx + 1 (respetando encabezado)
                ws.cell(row=r_idx + 1, column=c_idx, value=value)

        book.save(ruta)
        book.close()
        print("✅ Archivo actualizado correctamente con datos limpios y ordenados.")

    except Exception as e:
        print(f"❌ Error Excel: {e}")

# ==========================================
# 5. EJECUCIÓN
# ==========================================
if __name__ == "__main__":
    anio, mes = obtener_fecha_interactiva()

    if anio and mes:
        urls = generar_urls_dinamicas(anio, mes)
        datos_totales = []

        print("\n🚀 INICIANDO EXTRACCIÓN...")
        for i, url in enumerate(urls, 1):
            print(f"   ({i}/{len(urls)}) Leyendo...", end=" ")
            data = extraer_informacion(url)
            if data:
                print(f"✅ {data['Semana']}")
                datos_totales.append(data)
            else:
                print("❌ Falló")
            time.sleep(1)

        if datos_totales:
            guardar_excel_limpio(datos_totales, RUTA_ARCHIVO)
            print("\n🎉 FASE 1 COMPLETADA.")
        else:
            print("\n⚠️ No se encontraron datos.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📅 CONFIGURACIÓN DE BÚSQUEDA
----------------------------
👉 Ingresa el AÑO (ej. 2026): 2026
👉 Ingresa el MES (1-12): 1

🔍 Calculando semanas para: 1/2026...

🚀 INICIANDO EXTRACCIÓN...
   (1/4) Leyendo... ✅ 5-11 DE ENERO
   (2/4) Leyendo... ✅ 12-18 DE ENERO
   (3/4) Leyendo... ✅ 19-25 DE ENERO
   (4/4) Leyendo... ✅ 26 DE ENERO A 1 DE FEBRERO

💾 Actualizando archivo: Programación VMC_Septiembre-2025-2026.xlsx...


# 🚀 FASE 2: Fusión y Generación del Formato

In [ ]:
# =============================================================================
#  🚀 ASISTENTE DE PROGRAMACIÓN VMC - FASE 2: FUSIÓN Y GENERACIÓN (UNIFICADO)
# =============================================================================
#  CORRECCIÓN IMPORTANTE:
#  Ahora lee TANTO 'Info-reunion' COMO 'BD ASIG' para que no falte ningún dato.
#  - Datos Espirituales vienen de 'Info-reunion'.
#  - Nombres de Hermanos vienen de 'BD ASIG'.
# =============================================================================
# =============================================================================
#  VERSIÓN INTELIGENTE:
#  Detecta automáticamente el archivo Excel en la carpeta correcta para evitar
#  errores por tildes o nombres mal escritos.
# =============================================================================

import pandas as pd
from openpyxl import load_workbook
import re
import os
import sys
import glob

# ## 1. CONFIGURACIÓN
# =================================================================
CARPETA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC'

HOJA_INFO = 'Info-reunion'
HOJA_ASIG = 'BD ASIG'
HOJA_DESTINO = 'Formato'
ESPACIADO_FILAS = 8

# MAPEO MAESTRO (Aquí le decimos qué columna va a qué celda)
MAPEO_CELDAS = {
    # --- DESDE INFO-REUNION (Datos Espirituales) ---
    'Semana': 'B2',
    'Libro': 'I2',
    'Canción Inicial': 'D3',
    'Tesoros de la Biblia': 'C5',       # Título del discurso (Info-reunion)
    'Info Lectura Biblia': 'H7',
    'Segunda Canción': 'D14',
    'Tercera Canción': 'D19',
    'Info Estudio Libro': 'J17',
    'Maestros Título 1': 'C9', 'Maestros Título 2': 'C10',
    'Maestros Título 3': 'C11', 'Maestros Título 4': 'C12',
    'NVC Título 1': 'C15', 'NVC Título 2': 'C16', 'NVC Título 3': 'C17',

    # --- DESDE BD ASIG (Nombres de Hermanos) ---
    'PRESIDENCIA': 'O2',
    'ORACIÓN': 'O3',

    # ¡AQUÍ ESTÁN LOS CAMPOS QUE FALTABAN!
    'TESOROS DE LA BIBLIA': 'O5',       # Nombre del hermano (BD ASIG)
    'BUSQUEMOS PERLAS ESCONDIDAS': 'O6',# Nombre del hermano (BD ASIG)

    'LECTURA DE LA BIBLIA': 'O7',       # Nombre del Estudiante
    'SMM ASIG 1 ESTUD': 'L9', 'SMM ASIG 1 ACOMP': 'O9',
    'SMM ASIG 2 ESTUD': 'L10', 'SMM ASIG 2 ACOMP': 'O10',
    'SMM ASIG 3 ESTUD': 'L11', 'SMM ASIG 3 ACOMP': 'O11',
    'SMM ASIG 4 ESTUD': 'L12', 'SMM ASIG 4 ACOMP': 'O12',
    'NVC PARTE 1': 'O15', 'NVC PARTE 2': 'O16',
    'ESTUDIO LIBRO': 'O17', 'LECTOR LIBRO': 'O18', 'ORACIÓN FINAL': 'O19'
}

# ## 2. BUSCADOR INTELIGENTE
# =================================================================
def encontrar_archivo_correcto():
    print(f"🔍 Buscando archivo maestro en: {CARPETA_BASE}")
    patron = os.path.join(CARPETA_BASE, "*.xlsx")
    archivos = glob.glob(patron)
    archivos = [f for f in archivos if not os.path.basename(f).startswith("~$")]

    if not archivos:
        raise FileNotFoundError("❌ No hay archivos Excel en la carpeta.")

    archivo_maestro = None
    for archivo in archivos:
        nombre = os.path.basename(archivo)
        if nombre.startswith("Info_Reunion"): continue # Ignoramos el de solo info

        try:
            wb = load_workbook(archivo, read_only=True)
            if HOJA_ASIG in wb.sheetnames:
                print(f"✅ ¡Encontrado! Usando: '{nombre}'")
                archivo_maestro = archivo
                wb.close()
                break
            wb.close()
        except: continue

    if not archivo_maestro:
        raise FileNotFoundError(f"❌ Ningún archivo tiene la hoja '{HOJA_ASIG}'.")
    return archivo_maestro

# ## 3. LÓGICA DE COPIA
# =================================================================
def copiar_filas_unificadas(archivo_excel, filtro=None, num_filas=None, espaciado=8):
    try:
        print(f"\n{'='*60}\n🚀 INICIANDO FUSIÓN\n{'='*60}")

        # 1. LEER DATOS
        print(f"1️⃣ Leyendo hojas...")
        df_info = pd.read_excel(archivo_excel, sheet_name=HOJA_INFO)
        df_asig = pd.read_excel(archivo_excel, sheet_name=HOJA_ASIG)
        print(f"   ✅ Info-reunion: {len(df_info)} filas.")
        print(f"   ✅ BD ASIG:      {len(df_asig)} filas.")

        # 2. NORMALIZAR LLAVE
        df_info['Clave'] = df_info['Semana'].astype(str).str.upper().str.strip()
        df_asig['Clave'] = df_asig['SEMANA'].astype(str).str.upper().str.strip()

        # 3. FUSIÓN
        # Nota: Info usa 'Tesoros de la Biblia' (Título) y Asig usa 'TESOROS DE LA BIBLIA' (Nombre).
        # Al ser mayúsculas/minúsculas diferentes, Pandas las trata como columnas distintas. ¡Perfecto!
        df_completo = pd.merge(df_info, df_asig, on='Clave', how='left', suffixes=('', '_asig'))
        if 'SEMANA' in df_completo.columns:
             df_completo['SEMANA'] = df_completo['SEMANA'].fillna(df_completo['Semana'])

        # 4. FILTROS
        df_filtrado = df_completo.copy()
        if filtro:
            col, val = filtro
            val = str(val).upper().strip()
            print(f"\n2️⃣ Filtrando: {col} = '{val}'")
            if col in df_filtrado.columns:
                df_filtrado = df_filtrado[df_filtrado[col].astype(str).str.upper().str.strip() == val]
            else:
                print(f"   ⚠️ Columna '{col}' no encontrada. Verifica el nombre en BD ASIG.")

        if num_filas:
            print(f"\n3️⃣ Seleccionando últimas {num_filas} filas...")
            df_filtrado = df_filtrado.tail(num_filas)

        if df_filtrado.empty:
            print("⚠️ No hay datos para procesar.")
            return

        print(f"   ✅ Semanas a procesar: {len(df_filtrado)}")

        # 5. ESCRITURA
        print("\n4️⃣ Escribiendo en 'Formato'...")
        wb = load_workbook(filename=archivo_excel)
        hoja_dest = wb[HOJA_DESTINO]

        def extraer_coords(celda):
            match = re.match(r'([A-Z]+)(\d+)', celda.upper())
            return match.group(1), int(match.group(2))

        for idx, (_, fila) in enumerate(df_filtrado.iterrows()):
            desplazamiento = idx * (espaciado + 18)
            print(f"   📝 Escribiendo: {fila.get('Semana', '?')}")

            for col_orig, celda_base in MAPEO_CELDAS.items():
                # Verificamos si la columna existe y tiene datos
                if col_orig in fila.index and pd.notna(fila[col_orig]):
                    l, n = extraer_coords(celda_base)
                    hoja_dest[f"{l}{n + desplazamiento}"] = fila[col_orig]

        wb.save(archivo_excel)
        wb.close()
        print(f"\n🎉 ¡LISTO! Archivo actualizado correctamente.")

    except Exception as e:
        print(f"\n❌ ERROR: {e}")
        import traceback
        traceback.print_exc()

# ## 4. MENÚ
# =================================================================
def ejecutar_proceso_interactivo():
    if 'google.colab' in sys.modules:
        from google.colab import drive
        drive.mount('/content/drive')

    try:
        archivo_real = encontrar_archivo_correcto()
    except Exception as e:
        print(f"\n❌ {e}")
        return

    while True:
        print(f"\n{'='*60}")
        print(f"TRABAJANDO CON: {os.path.basename(archivo_real)}")
        print("1. Por MES (ej: 'DICIEMBRE')")
        print("2. Últimas 'N' semanas")
        print("3. TODO")
        print("4. Salir")
        op = input("\n👉 Opción: ")

        if op == '1':
            mes = input("   Mes: ")
            copiar_filas_unificadas(archivo_real, filtro=('MES', mes), espaciado=ESPACIADO_FILAS)
            break
        elif op == '2':
            try:
                n = int(input("   Cantidad: "))
                copiar_filas_unificadas(archivo_real, num_filas=n, espaciado=ESPACIADO_FILAS)
            except: print("Error numérico")
            break
        elif op == '3':
            copiar_filas_unificadas(archivo_real, espaciado=ESPACIADO_FILAS)
            break
        elif op == '4': break

if __name__ == "__main__":
    ejecutar_proceso_interactivo()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🔍 Buscando archivo maestro en: /content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC
✅ ¡Encontrado! Usando: 'Programación VMC_Septiembre-2025-2026.xlsx'

TRABAJANDO CON: Programación VMC_Septiembre-2025-2026.xlsx
1. Por MES (ej: 'DICIEMBRE')
2. Últimas 'N' semanas
3. TODO
4. Salir

👉 Opción: 2
   Cantidad: 5

🚀 INICIANDO FUSIÓN
1️⃣ Leyendo hojas...
   ✅ Info-reunion: 5 filas.
   ✅ BD ASIG:      18 filas.

3️⃣ Seleccionando últimas 5 filas...
   ✅ Semanas a procesar: 5

4️⃣ Escribiendo en 'Formato'...
   📝 Escribiendo: 1-7 DE DICIEMBRE
   📝 Escribiendo: 8-14 DE DICIEMBRE
   📝 Escribiendo: 15-21 DE DICIEMBRE
   📝 Escribiendo: 22-28 DE DICIEMBRE
   📝 Escribiendo: 29 DE DICIEMBRE DE 2025 A 4 DE ENERO DE 2026

🎉 ¡LISTO! Archivo actualizado correctamente.
